In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import csv
import os

import numpy as np
import torch.optim as optim

from pathlib import Path
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import lr_scheduler

from hw_grapheme.train import generate_stratified_k_fold_index, train_model
from hw_grapheme.utils import load_model_weight
from hw_grapheme.data_pipeline import create_dataloaders, load_data
from hw_grapheme.model.se_resnext50 import se_resnext50 
from hw_grapheme.loss_func import Loss_combine

# from torchtools.optim import RangerLars, RAdam
# from one_cycle import OneCycleLR
from torch.optim import Optimizer

# from warmup_scheduler import GradualWarmupScheduler

In [3]:
# load data 
pickle_paths = [
    "../data/processed_data/size_224/train_data_0.pickle",
    "../data/processed_data/size_224/train_data_1.pickle",
    "../data/processed_data/size_224/train_data_2.pickle",
    "../data/processed_data/size_224/train_data_3.pickle",
]

image_data, name_data, label_data = load_data(pickle_paths)

Load data done, shape: (200840, 224, 224), (200840,), (200840, 3)


In [4]:
configs = {
    "name": 'se-resnext50 baseline',
    "model": "se-resnext50",
    "pretrain": False,
    "head_info": "1 fc",
    "input_size": "224X224",
    "optimizer": "adam",
    "image_processing": "rotate(-10,10), scale(1.0, 1.15)",
    'batch_size': 64,
    'num_workers': 8,
    'pin_memory': True,
    'n_epoch': 48,
    'n_splits': 5,
    'random_seed': 2020,
    'mix_precision': False
}

In [5]:
batch_size = configs['batch_size']
num_workers = configs['num_workers'] 
pin_memory = configs['pin_memory']
n_epoch = configs['n_epoch']
n_splits = configs['n_splits']
random_seed = configs['random_seed']
mixed_precision = configs['mix_precision']

train_idx_list, test_idx_list = generate_stratified_k_fold_index(
    image_data, label_data, n_splits, random_seed
)

# create loss function
# criterion = nn.CrossEntropyLoss()
# criterion = Loss_combine()

# for discriminative lr
# my_list = ['module._fc.weight', 'module._fc.bias']
# params = list(filter(lambda kv: kv[0] in my_list, eff_b0.named_parameters()))
# base_params = list(filter(lambda kv: kv[0] not in my_list, eff_b0.named_parameters()))
# params = [kv[1] for kv in params]
# base_params = [kv[1] for kv in base_params]

# create data_transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=10, scale=(1.0, 1.15)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        # transforms.Normalize([0.0692], [0.2051]),
        # transforms.ToPILImage(),
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        # transforms.Normalize([0.0692], [0.2051])
    ]),
}

StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)


In [6]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%Y%m%d_%H%M%S")
current_time

'20200211_143956'

In [7]:
# import os
# import wandb
# # os.environ['WANDB_NOTEBOOK_NAME'] = 'model_3_efficient_net'
# %env WANDB_NOTEBOOK_NAME=model_3_efficient_net

In [10]:
pretrain = configs['pretrain']
criterion = Loss_combine()

for i, (train_idx, valid_idx) in enumerate(zip(train_idx_list, test_idx_list)):
    if i != 0:
        continue
        
    print(f"Training fold {i}")
    MODEL_DIR = Path(f"../model_weights/seresnext50_baseline/fold_{i}")
    MODEL_DIR.mkdir(exist_ok=True)
    
    # create model
    model = se_resnext50()

    if mixed_precision:
        model = apex.parallel.DistributedDataParallel(model)
        model.to("cuda")
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[0, 1], output_device=0
        )
        model, optimizer_ft = amp.initialize(
            model, optimizer_ft, opt_level="O1"
        )
    else:
        model.to("cuda")
        model = nn.DataParallel(model)
        # Add W&B logging

    # create optimizer
    optimizer_ft = optim.Adam(model.parameters(), weight_decay=1e-5)

    # create data loader
    data_loaders = create_dataloaders(
        image_data, name_data, label_data, train_idx, valid_idx,
        data_transforms, batch_size, num_workers, pin_memory
    )
    
    callbacks = {}
    callbacks = train_model(
        model, criterion, optimizer_ft, data_loaders,
        mixed_precision, callbacks, num_epochs=n_epoch,
        epoch_scheduler=None, save_dir=MODEL_DIR
    )

Training fold 0
Creating train dataloader...
Creating test dataloader...
Epoch 0/47
----------


Exception ignored in: <function _DataLoaderIter.__del__ at 0x7fde2c2048c0>
Traceback (most recent call last):
  File "/home/hugo/anaconda3/envs/kaggle/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/hugo/anaconda3/envs/kaggle/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/hugo/anaconda3/envs/kaggle/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/hugo/anaconda3/envs/kaggle/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/home/hugo/anaconda3/envs/kaggle/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
save_root_dir = Path("../model_weights/eff_0_baseline")
save_root_dir.mkdir(exist_ok=True)


config_save_path = save_root_dir/"config.csv"

with open(config_save_path, "w") as f:
    for key in configs.keys():
        f.write(f"{key},{configs[key]}\n")